In [ ]:
using CSV, Plots, DataFrames, Statistics, DelimitedFiles, LinearAlgebra
using Random, SparseArrays, ScikitLearn, LowRankModels
include("proxgrad.jl")

In [ ]:
Random.seed!(13)

In [ ]:
df = CSV.read("USwines.csv");
df = df[2:end];

In [ ]:
feature_names = names(df)

In [ ]:
size(df)

## Train Test Split

In [ ]:
df = df[.!(ismissing.(df[!, :price])), :]; # let's only consider the examples for which the price is known
df = df[shuffle(1:end), :] # we shuffle the data so that our train/test split will be truly random

train_proportion = 0.8
n = size(df, 1)
println("Size of dataset: ", string(n))
# The first t examples will form our training set; the rest will be our test set
t = convert(Int, round(train_proportion*n))

target = df[:, :price]
data = df[:, filter(col -> (col != :price), feature_names)]

train_x = data[1:t, :]
test_x = data[t:end, :]
train_y = target[1:t, :]
test_y = target[t:end, :]

In [ ]:
# This function just computes the mean squared error.
function MSE(y, pred)
    sum((y - pred).^2)/size(y, 1)
end

"""This function plots the main diagonal; 
for a "predicted vs true" plot with perfect predictions,
all data lies on this line"""
function plotDiagonal(xmin, xmax)
    xsamples = [xmin, xmax]
    plot!(xsamples, xsamples, color=:black)
end

"""This helper funciton plots x vs, y and labels the axes."""
function plotdata(x,y,xname, yname; margin=.05, plotDiag=true, zeromin=false)
    scatter(x,y, label="data")
    xlabel!(xname)
    ylabel!(yname)
    range_y = maximum(y) - minimum(y)
    range_x = maximum(x) - minimum(x)
    if plotDiag
        plotDiagonal(minimum(x)-margin*range_x, maximum(x)+margin*range_x)
    end
    if zeromin
        ylims!((0.0,maximum(y)+margin*range_y))
        xlims!((0.0,maximum(x)+margin*range_x))
    else
        ylims!((minimum(y)-margin*range_y,maximum(y)+margin*range_y))
        xlims!((minimum(x)-margin*range_x,maximum(x)+margin*range_x))
    end
end


"""This function plots the predicted labels vs the actual labels
(We only plots the first 1000 points to avoid slow plots.)"""
function plot_pred_true(test_pred, test_y, max_points = 1000)
    plotdata(test_pred[1:max_points], test_y[1:max_points], "Predicted (\$)", "True (\$)", zeromin=true)
end

In [ ]:
train_x = convert(Matrix, train_x); 
test_x = convert(Matrix, test_x); 

train_y = convert(Matrix, train_y);
test_y = convert(Matrix, test_y);

## linear

In [ ]:
train_x_1 = [ones(size(train_x)[1]) train_x];
test_x_1 = [ones(size(test_x)[1]) test_x];

In [ ]:
w_lin = train_x_1 \ train_y;

In [ ]:
train_pred = train_x_1 * w_lin; 
test_pred = test_x_1 * w_lin;
train_MSE = MSE(train_y, train_pred); 
test_MSE = MSE(test_y, test_pred);
println("Train MSE\t", train_MSE) 
println("Test MSE \t", test_MSE)

## ridge

In [ ]:
w_ridge = (train_x'*train_x + I) \ (train_x'*train_y)

In [ ]:
train_pred_ridge = train_x * w_ridge; 
test_pred_ridge = test_x * w_ridge;
train_MSE_ridge = MSE(train_y, train_pred_ridge); 
test_MSE_ridge = MSE(test_y, test_pred_ridge);
println("Train MSE\t", train_MSE) 
println("Test MSE \t", test_MSE)

## Text, PCA

In [ ]:
df_tf = CSV.read("message_embeddings.csv")

In [ ]:
train_all = join(train_x, df_tf, on=:id, kind=:left);
test_all = join(test_x, df_tf, on=:id, kind=:left);

In [ ]:
print(size(train_x))
train_embed = convert(Matrix, train_all[:, 94:end]);
test_embed = convert(Matrix, test_all[:, 94:end]);

## Training on all features, linear model

In [ ]:
train_embed_only = hcat(train_embed, ones(size(train_embed, 1)))
test_embed_only = hcat(test_embed, ones(size(test_embed, 1)))

@time begin
w_em = train_embed_only\train_y
end
    
train_pred = train_embed_only*w_em
test_pred = test_embed_only*w_em

train_MSE = MSE(train_y, train_pred)
test_MSE = MSE(test_y, test_pred)

println("Train MSE\t", string(train_MSE))
println("Test MSE \t", string(test_MSE))

In [ ]:
size(train_embed_only)

## PCA for text dimensionality reduction

In [ ]:
U, S, V = svd(train_embed_only);

Plot decay of singular values

In [ ]:
k_to_plot = 50

scatter(1:k_to_plot, S[1:k_to_plot], label="singular values")
xlabel!("singular value index")
ylabel!("singular values")

Plot explained variance (the sum of first several singular values over the sum of all singular values):

In [ ]:
k_to_plot = 400

scatter(1:k_to_plot, [sum(S[1:i]) / sum(S) for i=1:k_to_plot])
xlabel!("singular value index")
ylabel!("explained variance")

We loop over rank k from 10 to 100 and take a look at how training and test errors change. The following block may take a really long time, thus you may run the code in two blocks after the following block to quickly load results from previous executions.

In [ ]:
train_MSE_pca_all = []
test_MSE_pca_all = []

println("Train MSE of full dataset \t", string(train_MSE))
println("Test MSE of full dataset \t", string(test_MSE))

for k in 10:10:200
    println(k)
    
    Uk = U[:, 1:k];
    Sk = S[1:k];
    Vk = V[:, 1:k];
    
    train_embed_pca =  train_embed_only * Vk
    test_embed_pca = test_embed_only * Vk

    w_em_pca = train_embed_pca \ train_y

    train_pread_pca = train_embed_pca * w_em_pca
    test_pred_pca = test_embed_pca * w_em_pca

    train_MSE_pca = MSE(train_y, train_pred_pca)
    test_MSE_pca = MSE(test_y, test_pred_pca)

    println("Train MSE\t", string(train_MSE_pca))
    println("Test MSE \t", string(test_MSE_pca))
    
    append!(train_MSE_pca_all, train_MSE_pca)
    append!(test_MSE_pca_all, test_MSE_pca)    
    
end

save results

In [ ]:
writedlm("train_MSE_pca_all.txt", train_MSE_pca_all)
writedlm("test_MSE_pca_all.txt", test_MSE_pca_all)

Read previously saved results from local:

In [ ]:
train_MSE_pca_all = readdlm("train_MSE_pca_all.txt");
test_MSE_pca_all = readdlm("test_MSE_pca_all.txt");

Plot the training and test errors of fitting the linear regression model on both the entire dataset and the dimensionality reduced dataset:

In [ ]:
a = scatter(10:10:200, train_MSE_pca_all, label="training MSE")
b = scatter!(a, 10:10:200, test_MSE_pca_all, label="test MSE")
c = plot!(b, 10:200, [train_MSE for i=10:200], label="original training MSE")
d = plot!(c, 10:200, [test_MSE for i=10:200], label="original test MSE")
xlabel!("rank")
ylabel!("MSE")
title!("total number of features: 4689")

Note that this embedding provides a lot of information, but we are massively overfitting. This is to be expected: after all we have 4689 4689 parameters to fit, but only 20000 20000 training points. This makes PCA helpful for addressing overfitting.